# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [2]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [3]:
# Делаю pulled OLS ради интереса
pulled_ols = feols(log(v) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
pulled_ols %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           8.748607   0.301364 29.030005  < 2.2e-16
log(deaths_i)                         0.325192   0.090876  3.578422 0.00044799
log(deaths_j)                         0.150638   0.065264  2.308131 0.02216840
C8EV_International_travel_controls_i  0.234891   0.157781  1.488711 0.13837363
C8EV_International_travel_controls_j -0.020201   0.137475 -0.146944 0.88334648
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        *  
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.09796   Adj. R2: 0.034332

In [34]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i ^ j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i^j: 20,608,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error  t value Pr(>|t|)    
log(deaths_i)                        -0.015511   0.007764 -1.99787 0.047289 *  
log(deaths_j)                        -0.007561   0.007003 -1.07965 0.281790    
C8EV_International_travel_controls_i -0.038251   0.019968 -1.91564 0.057051 .  
C8EV_International_travel_controls_j -0.027171   0.020064 -1.35425 0.177411    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.702574     Adj. R2: 0.955423
                 Within R2: 8.835e-4

In [35]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i ^ j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i^j: 20,608
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                      -0.063500   0.092210 -0.688639
year2022                                       0.002725   0.108219  0.025184
year2020:log(deaths_i)                        -0.018044   0.008895 -2.028577
year2021:log(deaths_i)                        -0.016126   0.008817 -1.828983
year2022:log(deaths_i)                        -0.016141   0.010470 -1.541676
year2020:log(deaths_j)                        -0.018412   0.009123 -2.018171
year2021:log(deaths_j)                         0.001634   0.009407  0.173663
year2022:log(deaths_j)                         0.005853   0.010745  0.544703
year2020:C8EV_International_travel_controls_i -0.080207   0.030382 -2.639977
year2021:C8EV_International_travel_controls_i -0.031253   0.019420 -1.609341
year2022:C8EV_International_travel_

In [36]:
# С кластеризацией ошибок есть значимый эффект и наблюдается гетерогенность во времени (+ направление связи соотносится с описанным в литературе)
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i+j, signif.code=signif.code)
# А чем такая кластеризация отличается от i^j?

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,8.749*** (0.3014),,
4,log(deaths_i),0.3252*** (0.0909),-0.0155* (0.0078),
5,log(deaths_j),0.1506* (0.0653),-0.0076 (0.0070),
6,C8EV_International_travel_controls_i,0.2349 (0.1578),-0.0382. (0.0200),
7,C8EV_International_travel_controls_j,-0.0202 (0.1375),-0.0272 (0.0201),
8,year2021,,,-0.0635 (0.0922)
9,year2022,,,0.0027 (0.1082)


### Инструментальные переменные на панельных данных

In [37]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
pulled_tsls = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) 
 ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
population_density + population_density_j
  ,
 data = df
 )
pulled_tsls %>% summary(vcov = ~i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           7.558642   0.555417 13.608954 < 2.2e-16
fit_log(deaths_i)                    -0.187273   0.569224 -0.328997   0.74255
fit_log(deaths_j)                    -0.373980   0.617327 -0.605806   0.54543
C8EV_International_travel_controls_i  0.319239   0.239136  1.334971   0.18363
C8EV_International_travel_controls_j  0.037631   0.227472  0.165429   0.86880
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_

In [38]:
# Только FE года (инструменты эндогенные)
fe_iv_year = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j 
+
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
  population_density + population_density_j
 ,
 data = df
 )
fe_iv_year %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                     Estimate Std. Error t value   Pr(>|t|)    
fit_log(deaths_i)                    1.016266   0.148505 6.84330 1.2697e-10 ***
fit_log(deaths_j)                    0.624096   0.115088 5.42276 1.9388e-07 ***
C8EV_International_travel_controls_i 1.204183   0.270775 4.44718 1.5470e-05 ***
C8EV_International_travel_controls_j 0.833436   0.201166 4.14302 5.3404e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.27872     Adj. R2: -0.051569
                Within R2: -0.051726
F-test (1st stage), log(deaths_i): stat = 7,515.0, p < 2.2e-16, on 6 and 56,392 DoF.
F-test (1st stage), lo

In [39]:
feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i^j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.041419   0.031374 -1.320168 0.188513
fit_log(deaths_j)                    -0.006355   0.029966 -0.212090 0.832285
C8EV_International_travel_controls_i -0.049724   0.024681 -2.014698 0.045476
C8EV_International_travel_controls_j -0.025527   0.020830 -1.225487 0.222048
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i *  
C8EV_International_travel_controls_j   

In [45]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
fe_iv = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i^j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
# C1M_School_closing_i+C1M_School_closing_j
# +
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/2,257).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                     0.013198   0.039977  0.330131  0.74170 
fit_log(deaths_j)                     0.019157   0.028555  0.670870  0.50319 
C8EV_International_travel_controls_i -0.028114   0.024680 -1.139166  0.25620 
C8EV_International_travel_controls_j -0.015314   0.021951 -0.697624  0.48634 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.703126     Adj. R2:  0.955255
                 Within R2: -0.001143
F-test (1st stage), log(deaths_i): stat = 330.8    , p < 2.2e-16 , on 4 and 35,790 DoF.
F-test (1st stage), log(deaths_j): stat = 358.4    , p < 2.

In [46]:
fe_iv_age = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i^j|
 log(deaths_i) + log(deaths_j) ~ 
median_age_i + median_age_j
 ,
 data = df
 )
fe_iv_age %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/47).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j
Second stage: Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: year: 3,  i^j: 20,608
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                     0.022034   0.046722  0.471598  0.63780 
fit_log(deaths_j)                     0.026544   0.028271  0.938910  0.34908 
C8EV_International_travel_controls_i -0.023805   0.025684 -0.926835  0.35530 
C8EV_International_travel_controls_j -0.012833   0.020749 -0.618492  0.53706 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.703764     Adj. R2:  0.955272
                 Within R2: -0.002504
F-test (1st stage), log(deaths_i): stat = 611.6    , p < 2.2e-16 , on 2 and 36,127 DoF.
F-test (1st stage), log(deaths_j): stat = 666.0    , p < 2.2e-16 , on 2 and 36,127 DoF.
               

In [47]:
fe_iv_age_temp = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i^j|
 log(deaths_i) + log(deaths_j) ~ 
median_age_i + median_age_j
+
average_surface_temperature_i + average_surface_temperature_j 
 ,
 data = df
 )
fe_iv_age_temp %>% summary(cluster = ~ i+j)

NOTE: 17,245 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/8,783).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v)
Observations: 53,062
Fixed-effects: year: 3,  i^j: 19,210
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                     0.017085   0.047429  0.360231  0.71914 
fit_log(deaths_j)                     0.029085   0.028332  1.026576  0.30612 
C8EV_International_travel_controls_i -0.024841   0.025816 -0.962238  0.33734 
C8EV_International_travel_controls_j -0.010740   0.021333 -0.503453  0.61532 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.697462     Adj. R2:  0.955493
                 Within R2: -0.002325
F-test (1st stage), log(deaths_i): stat = 306.2    , p < 2.2e-16 , on 4 and 33,846 DoF.
F-test (1st stage), log(deaths_j): stat =

In [48]:
iv_both_endo = feols(log(v) ~  1 | year+i^j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                         0.002597   0.037209  0.069792
fit_log(deaths_j)                         0.005924   0.036975  0.160225
fit_C8EV_International_travel_controls_i -0.152683   0.097821 -1.560837
fit_C8EV_International_travel_controls_j -0.019369   0.086120 -0.224905
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.94444 
fit_log(deaths_j)                         0.87289 
fit_C8EV_International_travel_controls_i  0.120

In [49]:
# Даже тест Вальда говорит, что ничего не значимо...
wald(iv_both_endo, keep = 'log|C8', vcov = ~i+j)

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 0.767096, p-value = 0.546455, on 4 and 35,790 DoF, VCOV: Clustered (i & j).

In [50]:
# Тут всё плохо в тесте Саргана
# Мы верим в синюю схему, поэтому такая спецификация бессмысленна. Утверждение в ВКР: если схема верна, то влияение отрицательное.
feols(log(v) ~  1 | year|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                         Estimate Std. Error  t value
fit_log(deaths_i)                        1.017800   0.165012 6.168040
fit_log(deaths_j)                        0.626120   0.130734 4.789287
fit_C8EV_International_travel_controls_i 0.200526   1.619977 0.123783
fit_C8EV_International_travel_controls_j 0.038021   1.173255 0.032407
                                           Pr(>|t|)    
fit_log(deaths_i)                        4.6996e-09 ***
fit_log(deaths_j)                        3.5657e-06 ***
fit_C8EV_International_travel_controls_i 9.0163e-01    


In [51]:
iv_both_endo_temperature = feols(log(v) ~  1 | year+i^j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
+
average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo_temperature %>% summary(cluster = ~ i+j)
wald(iv_both_endo_temperature, keep = 'log|C8', vcov = ~i+j)

NOTE: 17,571 observations removed because of NA and infinite values (IV: 13,568/15,000).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v)
Observations: 52,736
Fixed-effects: year: 3,  i^j: 19,208
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                        -0.039411   0.053465 -0.737125
fit_log(deaths_j)                         0.032362   0.037318  0.867213
fit_C8EV_International_travel_controls_i -0.079222   0.111158 -0.712696
fit_C8EV_International_travel_controls_j -0.053372   0.081279 -0.656655
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.46209 
fit_log(deaths_j)                   

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 0.838281, p-value = 0.50057, on 4 and 33,522 DoF, VCOV: Clustered (i & j).

In [52]:
# Как будто ограничения тоже эндогенные...
# Но в своей схеме я считаю, что они экзогенные, потому что обосновали схему.
# Если бы они были эндогенными, то наша оценка была бы смещена куда-то
iv_itc_endo = feols(log(v) ~  1 + log(deaths_i) + log(deaths_j) | year+i^j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_itc_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 6,067, IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.109243   0.074697 -1.462470
fit_C8EV_International_travel_controls_j -0.001104   0.063304 -0.017442
log(deaths_i)                            -0.022025   0.011323 -1.945050
log(deaths_j)                            -0.004603   0.008709 -0.528482
                                         Pr(>|t|)    
fit_C8EV_International_travel_controls_i 0.145416    
fit_C8EV_International_travel_controls_j 0.986104    
log(deaths_i)                            0.053381 .  
log(deaths_j) 

In [53]:
# Как будто ограничения тоже эндогенные...
m8 = feols(log(v) ~  1 | year+i^j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m8 %>% summary(cluster = ~ i+j)

NOTE: 10,661 observations removed because of NA values (IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 59,646
Fixed-effects: year: 3,  i^j: 21,418
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.171016   0.089467 -1.911504
fit_C8EV_International_travel_controls_j -0.011865   0.070332 -0.168705
                                         Pr(>|t|)    
fit_C8EV_International_travel_controls_i 0.057556 .  
fit_C8EV_International_travel_controls_j 0.866221    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.719076     Adj. R2:  0.953829
                 Within R2: -0.0052  
F-test (1st stage), C8EV_International_travel_controls_i: stat = 464.

In [54]:
m9 = feols(log(v) ~  1 | year+i^j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m9 %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                   Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i) -0.055536   0.039093 -1.420614  0.15722 
fit_log(deaths_j)  0.002794   0.032980  0.084726  0.93258 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.703585     Adj. R2:  0.955199
                 Within R2: -0.002449
F-test (1st stage), log(deaths_i): stat = 400.1    , p < 2.2e-16 , on 6 and 35,790 DoF.
F-test (1st stage), log(deaths_j): stat = 366.2    , p < 2.2e-16 , on 6 and 35,790 DoF.
                       Wu-Hausman: stat =   3.10761, p = 0.04472 , on 2 and 35,790 DoF.
                           Sargan: stat =  23.7    ,

In [55]:
# Слово влияние это экспертное мнение на основе методов
# Температура это ложная корреляция, не надо
# Колонка 4 --- у нас нет данных, чтобы оценить эту связь. Недоидентифицированная система (всё на всё влияет)
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i+j, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v),log(v),log(v),log(v)
2,,,,,,,
3,Constant,7.559*** (0.5554),,,,,
4,log(deaths_i),-0.1873 (0.5692),1.016*** (0.1485),0.0132 (0.0400),0.0026 (0.0372),-0.0394 (0.0535),-0.0220. (0.0113)
5,log(deaths_j),-0.3740 (0.6173),0.6241*** (0.1151),0.0192 (0.0285),0.0059 (0.0370),0.0324 (0.0373),-0.0046 (0.0087)
6,C8EV_International_travel_controls_i,0.3192 (0.2391),1.204*** (0.2708),-0.0281 (0.0247),-0.1527 (0.0978),-0.0792 (0.1112),-0.1092 (0.0747)
7,C8EV_International_travel_controls_j,0.0376 (0.2275),0.8334*** (0.2012),-0.0153 (0.0220),-0.0194 (0.0861),-0.0534 (0.0813),-0.0011 (0.0633)
8,Fixed-Effects:,-----------------,------------------,----------------,----------------,----------------,-----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [28]:
# Без инструментов.
m_2020 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 5,531).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 4,126).

NOTE: 3,911 observations removed because of NA and infinite values (RHS: 3,911).



OLS estimation, Dep. Var.: log(v)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          5.563712   0.202765 27.4392 < 2.2e-16 ***
log(deaths_i)                        0.372474   0.014981 24.8636 < 2.2e-16 ***
log(deaths_j)                        0.244586   0.014349 17.0458 < 2.2e-16 ***
C8EV_International_travel_controls_i 0.895261   0.056778 15.7678 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.680061   0.057191 11.8910 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 3.9653   Adj. R2: 0.064359

OLS estimation, Dep. Var.: log(v)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error  t value   Pr(>|t|)
(Intercept)                          4.294474   0.128804 33.34124  < 2.2e-16
log(deaths_i)                        0.291123   0.019402 15.00443  < 2.2e-16
log(deaths_j)                        0.111539   0.017035  6.54750 5.9973e-11
C8EV_International_travel_controls_i 1.147440   0.038908 29.49124  < 2.2e-16
C8EV_International_travel_controls_j 0.835839   0.039068 21.39448  < 2.2e-16
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        ***
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.03296   Adj. R2: 0.077936

OLS estimation, Dep. Var.: log(v)
Observations: 18,916
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                          8.909318   0.078977 112.80953  < 2.2e-16
log(deaths_i)                        0.483443   0.015579  31.03087  < 2.2e-16
log(deaths_j)                        0.258449   0.014147  18.26831  < 2.2e-16
C8EV_International_travel_controls_i 0.671115   0.072550   9.25034  < 2.2e-16
C8EV_International_travel_controls_j 0.365265   0.066691   5.47697 4.3819e-08
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        ***
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.07348   Adj. R2: 0.061527

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.564*** (0.2028),4.294*** (0.1288),8.909*** (0.0790)
4,log(deaths_i),0.3725*** (0.0150),0.2911*** (0.0194),0.4834*** (0.0156)
5,log(deaths_j),0.2446*** (0.0143),0.1115*** (0.0170),0.2584*** (0.0141)
6,C8EV_International_travel_controls_i,0.8953*** (0.0568),1.147*** (0.0389),0.6711*** (0.0726)
7,C8EV_International_travel_controls_j,0.6801*** (0.0572),0.8358*** (0.0391),0.3653*** (0.0667)
8,____________________________________,__________________,__________________,__________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedas.-rob.


In [29]:
# С инструментами оценки коэффициентов всё равно не похожи на панельные данные (пространственные смещённые, т.к должно быть много unobservables)
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 3,419, IV: 3,381/3,419).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 3,290, IV: 1,557/3,290).

NOTE: 4,249 observations removed because of NA and infinite values (RHS: 3,093, IV: 1,129/3,952).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          5.969386   0.213890 27.9086 < 2.2e-16 ***
fit_log(deaths_i)                    0.902226   0.022833 39.5137 < 2.2e-16 ***
fit_log(deaths_j)                    0.558973   0.020960 26.6685 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.056775   0.061515 17.1791 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.784651   0.060168 13.0409 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.16015   Adj. R2: -0.029856
F-test (1st stage), log(deaths_i): stat = 2,699.8, p < 2.2e-16, on 6 and 18,382 DoF.
F-te

TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          4.481759   0.137363 32.6271 < 2.2e-16 ***
fit_log(deaths_i)                    1.039850   0.029786 34.9101 < 2.2e-16 ***
fit_log(deaths_j)                    0.568131   0.026997 21.0446 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.146007   0.041462 27.6397 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.833670   0.041619 20.0308 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.31431   Adj. R2: -0.055203
F-test (1st stage), log(deaths_i): stat = 2,100.5, p < 2.2e-16, on 6 and 19,423 DoF.
F-te

TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 18,578
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)                          8.938381   0.082064 108.9203 < 2.2e-16 ***
fit_log(deaths_i)                    0.871966   0.020231  43.1007 < 2.2e-16 ***
fit_log(deaths_j)                    0.561645   0.019106  29.3965 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.289039   0.078296  16.4637 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.796659   0.073831  10.7904 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.18539   Adj. R2: 0.013015
F-test (1st stage), log(deaths_i): stat = 5,398.6, p < 2.2e-16 , on 6 and 18,569 DoF

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.969*** (0.2139),4.482*** (0.1374),8.938*** (0.0821)
4,log(deaths_i),0.9022*** (0.0228),1.040*** (0.0298),0.8720*** (0.0202)
5,log(deaths_j),0.5590*** (0.0210),0.5681*** (0.0270),0.5616*** (0.0191)
6,C8EV_International_travel_controls_i,1.057*** (0.0615),1.146*** (0.0415),1.289*** (0.0783)
7,C8EV_International_travel_controls_j,0.7847*** (0.0602),0.8337*** (0.0416),0.7967*** (0.0738)
8,____________________________________,__________________,__________________,__________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedas.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.969*** (1.265),4.482*** (0.7216),8.938*** (0.5137)
4,log(deaths_i),0.9022*** (0.1570),1.040*** (0.2066),0.8720*** (0.1270)
5,log(deaths_j),0.5590*** (0.1119),0.5681*** (0.1414),0.5616*** (0.1046)
6,C8EV_International_travel_controls_i,1.057* (0.4228),1.146*** (0.2648),1.289* (0.5608)
7,C8EV_International_travel_controls_j,0.7847** (0.2893),0.8337*** (0.1878),0.7967. (0.4445)
8,____________________________________,__________________,__________________,__________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
